In [1]:
! pip install langchain

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.21
    Uninstalling pydantic-1.10.21:
      Successfully uninstalled pydantic-1.10.21
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langserve 0.2.3 requires langchain-core<0.3,>=0.1, but you have langchain-core 0.3.40 which is incompatible.
myapp 0.1.0 requires pydantic<2, but you have pydantic 2.10.6 which is incompatible.


- 使用 LangChain 构建的许多应用程序将包含多个步骤，并多次调用 LLM 调用。随着这些应用程序变得越来越复杂，能够检查链或代理内部究竟发生了什么变得至关重要。最好的方法是使用LangSmith。

In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] ="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" #getpass.getpass()

In [3]:
! pip install -qU "langchain[openai]"

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="THUDM/glm-4-9b-chat",
    model_provider="openai",
    api_key="sk-pvdcqtmuphqjgepywixzapshexhxcugkhnrtjnuhdotlvgtx", 
    base_url="https://api.siliconflow.cn/v1",
    temperature=0.1,
    max_retries=3,
    
)

In [15]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Chinese:"),
    HumanMessage("hi!"),
]

model.invoke(messages)

AIMessage(content='\n嗨！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 17, 'total_tokens': 20, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'THUDM/glm-4-9b-chat', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-6ccf2bf4-4bf4-431c-be8f-223a3bcacfe4-0', usage_metadata={'input_tokens': 17, 'output_tokens': 3, 'total_tokens': 20, 'input_token_details': {}, 'output_token_details': {}})

In [16]:
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])

AIMessage(content='\nHello 👋! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 6, 'total_tokens': 18, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'THUDM/glm-4-9b-chat', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-cfdc1cce-e8f0-464b-b52f-e7acecaee49e-0', usage_metadata={'input_tokens': 6, 'output_tokens': 12, 'total_tokens': 18, 'input_token_details': {}, 'output_token_details': {}})

In [17]:
for token in model.stream(messages):
    print(token.content, end="|")

|
|嗨|！||

In [23]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following text to {language}:\n{text}"

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        # ("human", "{text}"),
    ]
)
promt = prompt_template.invoke({"text": "Hello world", "language": "chinese"})
promt.to_messages()

[SystemMessage(content='Translate the following text to chinese:\nHello world', additional_kwargs={}, response_metadata={})]

In [24]:
response= model.invoke(promt)
print(response.content)


你好，世界


#### 构建语义搜索引擎#

In [ ]:
from langchain_core.documents import Document

document = [
    Document(
        page_content="This is a test document. This is the second sentence.",
        metadata={"source": "test.txt", "page": 1},
    ),
    Document(
        page_content="This is another test document. This is the second sentence.",
        metadata={"source": "test.txt", "page": 2},
    ),
]

In [26]:
! pip install langchain-community pypdf

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [27]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example_data/基本RBRVS医院绩效考核系统.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

33


In [28]:
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

基于运营的RBRVS绩效体系
介绍
医改背景下医院内部激励机制的创新与应用
jackluo
2025/2/14

{'producer': '', 'creator': 'WPS 演示', 'creationdate': '2025-02-14T16:47:46+08:00', 'author': '', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2025-02-14T16:47:46+08:00', 'sourcemodified': "D:20250214164746+08'00'", 'subject': '', 'title': '', 'trapped': '/False', 'source': './example_data/基本RBRVS医院绩效考核系统.pdf', 'total_pages': 33, 'page': 0, 'page_label': '1'}


In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

33

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import init_embeddings

embeddings = init_embeddings(
    base_url="https://api.siliconflow.cn/v1/embeddings",
    api_key="sk-pvdcqtmuphqjgepywixzapshexhxcugkhnrtjnuhdotlvgtx",
    model="BAAI/bge-m3",
    provider="ollama"
)

# embeddings = OpenAIEmbeddings(
#     base_url=" https://api.siliconflow.cn/v1/embeddings",
#     api_key="sk-pvdcqtmuphqjgepywixzapshexhxcugkhnrtjnuhdotlvgtx",
#     model="BAAI/bge-m3"
# )

C:\Users\jackluo\AppData\Local\Temp\ipykernel_9832\551284539.py:4: LangChainBetaWarning: The function `init_embeddings` is in beta. It is actively being worked on, so the API may change.
  embeddings = init_embeddings(


In [37]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

NotFoundError: 404 page not found